In [1]:
import numpy as np
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment
from gillespy2 import SSACSolver

# devils_v1.0
- Need to validate timespan

In [2]:
class Devilsv101(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="devils_v1.0(1)")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="r", expression="1"))
        self.add_parameter(Parameter(name="K", expression="217"))
        self.add_parameter(Parameter(name="ds", expression="0.22"))
        self.add_parameter(Parameter(name="dI", expression="1.28"))
        self.add_parameter(Parameter(name="L", expression="0.75"))
#         self.add_parameter(Parameter(name="sigma", expression="0.5"))
#         self.add_parameter(Parameter(name="p", expression="0.6"))
        self.add_parameter(Parameter(name="k0", expression="4"))
#         self.add_parameter(Parameter(name="vf", expression="1"))
#         self.add_parameter(Parameter(name="vt", expression="0"))

        # Variables
        self.add_species(Species(name="S", initial_value=157, mode="discrete"))
        self.add_species(Species(name="E", initial_value=10, mode="discrete"))
        self.add_species(Species(name="I", initial_value=20, mode="discrete"))
#         self.add_species(Species(name="W", initial_value=0, mode="discrete"))
        self.add_species(Species(name="R", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Devils", initial_value=45021, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="birth", reactants={}, products={'Devils': 1, 'S': 1}, propensity_function="r*(S+E+I+R)*(1-(S+E+I+R)/K)"))
        self.add_reaction(Reaction(name="death_S", reactants={'Devils': 1, 'S': 1}, products={}, rate=self.listOfParameters["ds"]))
        self.add_reaction(Reaction(name="transmission", reactants={'S': 1}, products={'E': 1}, propensity_function="k0*S*I/(S+E+I+R)"))
        self.add_reaction(Reaction(name="death_E", reactants={'Devils': 1, 'E': 1}, products={}, propensity_function="ds*E"))
        self.add_reaction(Reaction(name="latency", reactants={'E': 1}, products={'I': 1}, propensity_function="E/L"))
#         self.add_reaction(Reaction(name="vaccine_failure", reactants={'W': 1}, products={'E': 1}, propensity_function="k0*sigma*W*I/(S+E+I+W+R)"))
        self.add_reaction(Reaction(name="death_I", reactants={'Devils': 1, 'I': 1}, products={}, propensity_function="dI*I"))
#         self.add_reaction(Reaction(name="death_W", reactants={'W': 1}, products={}, propensity_function="ds*W"))
        self.add_reaction(Reaction(name="death_R", reactants={'Devils': 1, 'R': 1}, products={}, propensity_function="ds*R"))

#         # Event Triggers
#         vaccination_trig = EventTrigger(expression="t >= vt+vf", initial_value=False, persistent=False)

#         # Event Assignments
#         vaccination_assign_1 = EventAssignment(variable="vt", expression="vt+1")
#         vaccination_assign_2 = EventAssignment(variable="W", expression="W+(S*p)")
#         vaccination_assign_3 = EventAssignment(variable="S", expression="S*(1-p)")

#         # Events
#         self.add_event(Event(name="vaccination", trigger=vaccination_trig, assignments=[vaccination_assign_1, vaccination_assign_2, vaccination_assign_3], delay=None, priority="0", use_values_from_trigger_time=True))

        # Timespan
        self.timespan(np.arange(0, 21, 1/12))

In [3]:
model = Devilsv101()
print(len(model.tspan))

312


# Simulation Parameters

In [ ]:
def configure_simulation():
    kwargs = {
        "solver":TauHybridSolver,
        "number_of_trajectories":100,
        "seed":None,
        "tau_tol":0.03,
        "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

# Model Inference

In [ ]:
%load_ext autoreload
%autoreload 2

from tsfresh.feature_extraction.settings import MinimalFCParameters
from sciope.utilities.priors import uniform_prior
from sciope.utilities.summarystats import auto_tsfresh
from sciope.utilities.distancefunctions import naive_squared
from sciope.inference.abc_inference import ABC
from sciope.inference.rep_smc_abc import ReplenishmentSMCABC
from sklearn.metrics import mean_absolute_error
from dask.distributed import Client

## Generate some fixed(observed) data based on default parameters of the model

In [39]:
import pandas as pd

confirmed_data = pd.read_csv('../Devil Dataset.csv')
devil_pop = np.array(confirmed_data['Population'].iloc[:].values)
obs = np.vstack([devil_pop]).reshape(1, 1, -1)
print(obs)

[[[45020.76062 45157.79794 45301.81398 45516.60821 45701.20098
   45826.87714 45998.33157 46216.37993 46372.02246 46508.16238
   46640.8784  46844.3869  46989.83142 47251.77616 47400.89737
   47704.96134 47906.61741 48105.94297 48361.13849 48504.52751
   48694.27106 48903.80591 49108.7615  49376.10766 49510.46679
   49615.38407 49640.5478  49808.41714 49864.12469 49898.22346
   49965.54229 50093.25281 50171.76771 50239.38827 50306.54096
   50387.03061 50511.36391 50593.04774 50739.25817 50874.16843
   51011.92355 51148.96077 51286.33891 51423.19183 51560.44053
   51703.76661 51895.1796  51951.0532  52023.60414 52091.16357
   52159.82513 52229.19536 52294.90024 52372.32169 52371.29875
   52502.86674 52509.8829  52640.72837 52615.69874 52675.29124
   52761.08243 52781.76918 52857.26476 52931.87579 52939.76047
   52999.71057 53069.14388 53132.45976 53137.02743 53217.84172
   53256.1655  53283.53619 53330.02521 53281.29293 53204.07263
   53190.94311 53195.21785 53128.49065 53065.94199 5306

## Define prior distribution

In [ ]:
# take default from mode 1 as reference
default_param = np.array(list(model.listOfParameters.items()))[:, 1]

bound = []
for exp in default_param:
    bound.append(float(exp.expression))

# Set the bounds
bound = np.array(bound)
dmin = bound * 0.1
dmax = bound * 2.0

# Here we use uniform prior
uni_prior = uniform_prior.UniformPrior(dmin, dmax)

## Define simulator

In [ ]:
def set_model_parameters(params, model):
    # params - array, need to have the same order as model.listOfParameters
    for e, pname in enumerate(model.listOfParameters.keys()):
        model.get_parameter(pname).set_expression(params[e])
    return model

# Here we use the GillesPy2 Solver
def simulator(params, model):
    model_update = set_model_parameters(params, model)

    res = model_update.run(**kwargs)
    res = res.to_array()
    tot_res = np.asarray([x.T for x in res]) # reshape to (N, S, T)
    # should not contain timepoints
    tot_res = tot_res[:, 1:, :]

    return tot_res

# Wrapper, simulator function to abc should should only take one argument (the parameter point)
def simulator2(x):
    return simulator(x, model=model)

## Define summary statistics and distance function

In [ ]:
# Function to generate summary statistics
summ_func = auto_tsfresh.SummariesTSFRESH()

# Distance
ns = naive_squared.NaiveSquaredDistance()

## Start local cluster using dask client

In [ ]:
c = Client()

## Start abc instance

In [ ]:
abc = ABC(fixed_data,
          sim=simulator2,
          prior_function=uni_prior,
          summaries_function=summ_func.compute,
          distance_function=ns
         )

In [ ]:
# First compute the fixed(observed) mean
abc.compute_fixed_mean(chunk_size=2)

In [ ]:
res = abc.infer(num_samples=100, batch_size=10, chunk_size=2)

In [ ]:
mae_inference = mean_absolute_error(bound, abc.results['inferred_parameters'])